### Initilize a chat model

The core parameters of a chat model class is:
- `temperature` : Control the randomness or creativity of the output
- `max tokens` : Maximum number of tokens to generate
- `timeout` : wait time for retying request.
- `max retires` : max number of times we should retry a request.

LangChain is model agnostic, you can use any class provided in the documentation that support other model providers. The chat_init can handle most popular models by just giving the name. Others might need to be imported from Lanchain Core. 

For any model you add the API keys in the .env file. 

In [1]:
from langchain_ollama import ChatOllama
model = ChatOllama(model="llama3.2:1b", 
                   temperature=0,
                   validate_model_on_init=True,
                   seed=42)

In [ ]:
for chunk in model.stream("Hello World!"):
    print(chunk.text, end = "" , flush=True)

Hello! It's great to see you're starting with "Hello World!" That's a classic and iconic message that many people remember from their early programming experiences. What would you like to talk about or do next?

In [ ]:
from langchain.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage("You are a poetry expert"),
    HumanMessage("Write a haiku about spring"),
    AIMessage("Cherry blossoms bloom...")
]
response = model.invoke(messages)

Usually it takes a lot of time for Agents (LLMs) to come up with response, it can be 10-20s. A way to reduce the "percived" latency is by streaming tokens to the user. Display tokens as they generate. This is done by a lot of popular model providers like ChatGPT.

In [3]:
from langchain.agents import create_agent
agent = create_agent(model)

In [5]:
from langchain.messages import HumanMessage

response = agent.invoke({'messages':[HumanMessage(content="What is the capital of the moon")]
})

response

{'messages': [HumanMessage(content='What is the capital of the moon', additional_kwargs={}, response_metadata={}, id='90640c74-27ac-4edc-b86a-b693224421b2'),
  AIMessage(content='There is no capital city on the Moon. The Moon is a natural satellite that orbits the Earth, and it does not have any permanent human settlements or cities. While there have been several manned missions to the Moon in recent years, including NASA\'s Apollo program, these missions were focused on exploration and scientific research, not settlement.\n\nThe United Nations has designated the Moon as a "natural satellite" rather than a planet, which means it is not considered a separate country with its own government. However, there are ongoing efforts to establish a sustainable human presence on the Moon in the near future, including plans for lunar bases and settlements.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-22T10:26:49.5985113Z', 'done': True, 'done_reason': '

In [7]:
from pprint import pprint
pprint(response)

# To access the latest AI response 
response['messages'][-1].content

{'messages': [HumanMessage(content='What is the capital of the moon', additional_kwargs={}, response_metadata={}, id='90640c74-27ac-4edc-b86a-b693224421b2'),
              AIMessage(content='There is no capital city on the Moon. The Moon is a natural satellite that orbits the Earth, and it does not have any permanent human settlements or cities. While there have been several manned missions to the Moon in recent years, including NASA\'s Apollo program, these missions were focused on exploration and scientific research, not settlement.\n\nThe United Nations has designated the Moon as a "natural satellite" rather than a planet, which means it is not considered a separate country with its own government. However, there are ongoing efforts to establish a sustainable human presence on the Moon in the near future, including plans for lunar bases and settlements.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-22T10:26:49.5985113Z', 'done': True, 'don

'There is no capital city on the Moon. The Moon is a natural satellite that orbits the Earth, and it does not have any permanent human settlements or cities. While there have been several manned missions to the Moon in recent years, including NASA\'s Apollo program, these missions were focused on exploration and scientific research, not settlement.\n\nThe United Nations has designated the Moon as a "natural satellite" rather than a planet, which means it is not considered a separate country with its own government. However, there are ongoing efforts to establish a sustainable human presence on the Moon in the near future, including plans for lunar bases and settlements.'

In [8]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is the capital of Moon"),
                  AIMessage(content="The capital of Moon is Crater Hole"),
                  HumanMessage(content="Intresting! Tell me more!")]}
)
pprint(response)

{'messages': [HumanMessage(content='What is the capital of Moon', additional_kwargs={}, response_metadata={}, id='559bb178-72a1-4652-9ac2-f01a95c7a5f2'),
              AIMessage(content='The capital of Moon is Crater Hole', additional_kwargs={}, response_metadata={}, id='06831d79-3d70-4a2a-8955-7ec1ae10f167'),
              HumanMessage(content='Intresting! Tell me more!', additional_kwargs={}, response_metadata={}, id='fb36386e-3c60-49fe-9846-0802c4703d74'),
              AIMessage(content="I'm glad you're interested. Unfortunately, I need to clarify that there are no human settlements or established governments on the Moon. The Moon is a natural satellite that orbits the Earth, and it's not capable of supporting life or hosting any permanent human populations.\n\nHowever, if we were to imagine a scenario where humans did establish a settlement on the Moon, Crater Hole might be a fascinating location. It's a relatively large crater with a diameter of about 1 kilometer (0.6 miles), whi

Streaming

In [11]:
for token, metadata in agent.stream(
   {"messages":[HumanMessage(content="Tell me all about Luna City")]},
   stream_mode="messages"
):
    
# token is a message chunk with token content
# metadata contains which node produced the token

    if token.content:
        print(token.content, end="", flush=True)

Luna City is a fictional city in the Harry Potter universe. It appears in J.K. Rowling's book "Harry Potter and the Order of the Phoenix" (2003) and later in "Harry Potter and the Deathly Hallows" (2007). The city is located on the outskirts of London, England.

Luna City is a hub for magical creatures, particularly those with a connection to the moon. It's a place where wizards and witches can come to learn about and interact with these creatures, as well as to trade goods and services. The city is also home to several magical shops and market stalls that cater to the needs of lunar creatures.

One of the most notable aspects of Luna City is its connection to the Patronus Charm, a powerful spell that can repel Dementors. According to legend, the city's founders were able to harness the power of the moon to create a protective barrier around the city, which has since become known as the "Lunar Barrier."

The city is also home to several magical creatures, including Hippogriffs, Thestra

Strcutured Outputs and System Prompt

Set a system prompt to narrow down the scope to LLM. Keeps a agent focused and have a better agent.

In [9]:
question = HumanMessage(content="What's the capital of the moon?")

system_prompt = "You are a science fiction writer, create a capital city at the users request."

scifi_agent = create_agent(
    model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

I think there may be some confusion here. There is no capital city on the Moon. The Moon is a natural satellite that orbits the Earth, and it does not have a permanent human settlement or government.

However, if you'd like, I can create a fictional capital city for your science fiction story. Here's an idea:

Name: Lunaria

Location: On the lunar surface, near the Sea of Tranquility (formerly known as Mare Tranquillitatis)

Lunaria is a futuristic, high-tech city that has been built by a coalition of space-faring nations and private corporations. The city is designed to be self-sustaining, with its own air, water, and food production systems. Its architecture is a blend of traditional lunar habitats and cutting-edge, modular construction techniques.

The city's government is known as the Lunar Assembly, which is composed of representatives from each member nation or corporation. The Assembly meets in Lunaria's central hub, a massive, geodesic dome that provides a stable and protected 